In [ ]:
import pandas as pd
import numpy as np

## load data

In [105]:
pain = pd.read_csv('data/pain.csv', sep =',', names = list(range(0,30)))
bowel_diseases = pd.read_csv('data/bowel diseases.csv', sep=',', names = list(range(0,30)))
joints = pd.read_csv('data/joints.csv', sep=',', names = list(range(0,30)))
psoriasis = pd.read_csv('data/psoriasis.csv', sep=',', names = list(range(0,30)))
sclerosis = pd.read_csv('data/sclerosis.csv', sep=',', names = list(range(0,30)))

In [108]:
pain = pain[[4,0,5,1,2,3,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]
pain[5] = pain[5].map(lambda x: x.lstrip('מספר תגובות -'))
pain.columns = ['url','data','number','name','title',1,2,3,1,2,3,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
pain.head(1)    

,url,data,number,name,title,1,2,3,1,2,...,15,16,17,18,19,20,21,22,23,24
0,https://www.camoni.co.il/411804/121662,02/01/2015,2,עסאף,כאב נירופטי ממקור לא ידוע,לפני שנה צינור ברזל נפל ממנוף ונפגעתי בראש עם ...,"XXלימור חורשXX\nשלום עסאף, קודם כל רק בריאות ו...","XXד""ר גור רותXX\nשלום עאטף,&nbsp;\r\n\r\nאנא פ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
